In [104]:
import pandas as pd
import os, fnmatch
import numpy as np
from scipy.stats import iqr
from numpy import percentile

def locate(pattern, root=os.curdir):
    '''Locate all files matching supplied filename pattern in and below
    supplied root directory.'''
    for path, dirs, files in os.walk(os.path.abspath(root)):
        for filename in fnmatch.filter(files, pattern):
            return os.path.join(path, filename)
            #yield os.path.join(path, filename)

In [2]:
root = '/net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/'

In [3]:
found_confounds = locate('sub-*confounds_regressors.tsv', root)
found_rsbold = locate('sub-*MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', root)

In [10]:
type(found_confounds)

str

In [5]:
found_rsbold

'/net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/sub-1001/ses-1/func/sub-1001_ses-1_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'

In [16]:
confounds_filename = found_confounds
#18 regressors are these 9, and their derivatives.
motion_reg_colnames = ['csf', 'white_matter', 'global_signal', 'trans_x', 'trans_y', 'trans_z', 'rot_x' , 'rot_y', 'rot_z']
#To compute spike regressors
spikereg_colnames = ['dvars', 'framewise_displacement']
deriv = True
quad = False

In [13]:
confounds_filename

'/net/holynfs01/srv/export/mclaughlin/share_root/stressdevlab/GenR_derivatives/fmriprep/sub-1001/ses-1/func/sub-1001_ses-1_task-rest_desc-confounds_regressors.tsv'

In [15]:
motion_reg_colnames + spikereg_colnames

['csf',
 'white_matter',
 'global_signal',
 'trans_x',
 'trans_y',
 'trans_z',
 'rot_x',
 'rot_y',
 'rot_z',
 'dvars',
 'framewise_displacement']

In [100]:
use_derivs = True
use_quads = False
dvar_reg = True
displace_reg = True

In [27]:
confounds = pd.read_csv(confounds_filename, sep = '\t').reindex(columns = motion_reg_colnames + spikereg_colnames)

In [133]:
confounds_motion_reg = confounds[motion_reg_colnames]
confounds_spikereg = confounds[spikereg_colnames]

if use_derivs == True:
    deriv = np.diff(confounds_motion_reg, n=1, axis=0)
    deriv_df = pd.DataFrame(np.insert(arr=confounds_deriv, obj=0, values=0, axis=0), 
                            columns=[s + '_deriv' for s in confounds_motion_reg.columns])
    confounds_motion_all = pd.concat([confounds_motion_reg, deriv_df], axis=1)
else:
    confounds_motion_all = confounds_motion_reg

if use_quads == True:
    quads_df = confounds_motion_all.apply(np.square)
    quads_df.columns = [s + '_sq' for s in quads_df.columns]
    confounds_motion_all = pd.concat([confounds_motion_all, quads_df], axis=1)

#Compute some more columns for the confounds df
#from https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FSLMotionOutliers:
#--thresh=<val>       specify absolute threshold value (otherwise use box-plot cutoff = P75 + 1.5*IQR)

spikes=pd.Series([0]*confounds_spikereg.shape[0])

if dvar_reg == False and displace_reg == False:
    raise Exception('Must include at least one spike regressor option.')

if dvar_reg == True:
    dvar_outlier_boundry = percentile(confounds_spikereg.loc[1:, 'dvars'], q = [75]) + 1.5*iqr(confounds_spikereg.loc[1:, 'dvars'])
    spikes = spikes + confounds_spikereg['dvars'].apply(lambda dvar: 1 if dvar > dvar_outlier_boundry else 0)
    

In [134]:
spikes

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
16     0
17     0
18     0
19     0
20     0
21     1
22     0
23     0
24     0
25     0
26     0
27     0
28     0
29     0
      ..
150    0
151    0
152    0
153    0
154    0
155    0
156    0
157    0
158    0
159    0
160    0
161    0
162    0
163    0
164    0
165    0
166    0
167    0
168    0
169    0
170    0
171    0
172    0
173    0
174    0
175    0
176    0
177    0
178    0
179    0
Length: 180, dtype: int64

In [ ]:
from nipype import Node, Workflow

# it's important to
# pass filenames to Nodes as absolute paths
from os.path import abspath
from nipype.interfaces import afni
import nibabel as nib


In [ ]:
in_file = abspath(found_rsbold)
number_trs = nib.load(in_file).shape[3]

In [ ]:
#set up the 3dDespike node, which will be connected to the 3dDeconvolve node
despike = Node(afni.Despike(in_file = in_file), name = "despike")
    

In [ ]:
#for getting the results of 1dBport into a data frame
import subprocess

In [ ]:
#create bandpass regressors
result = subprocess.run(['1dBport', '-nodata', str(number_trs), '2', '-band', '0.01', '.08', '-invert', '-nozero'], stdout=subprocess.PIPE)

In [ ]:
from io import StringIO

b = StringIO(result.stdout.decode('utf-8'))

df = pd.read_csv(b, sep='\s+', header=None)